In [4]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

from datetime import datetime, timedelta

In [11]:
door1 = pd.read_csv("data/data_door1.txt",  sep=";", header=None)
door1 = door1.set_axis(["Timestamp", "x", "y", "z", "pressure"], axis=1, inplace=False)

door2 = pd.read_csv("data/data_door2.txt",  sep=";", header=None)
# has 5 variables ???

testdata = pd.read_csv("data/testdata.csv", sep=";", header=None)
testdata = testdata.set_axis(["Timestamp", "x", "y", "z", "pressure"], axis=1, inplace=False)

In [6]:
### calculate medians

def calculateMedian(variable, size):
    listOfMedians = list()
    for i in range(0, len(variable)):
        start = max(i - size//2, 0) # max to avoid negatives
        end = min(i + (size-size//2), len(variable)) # evitar problemes amb parells

        calculate_median = variable[start:end]
        calculate_median.sort()
        median = calculate_median[len(calculate_median)//2]
        listOfMedians.append(median)
    return listOfMedians

In [12]:
door1["x"] = calculateMedian(list(door1["x"]), 100)
door1["y"] = calculateMedian(list(door1["x"]), 100)
door1["z"] = calculateMedian(list(door1["x"]), 100)

### export

In [14]:
from datetime import datetime, timedelta

# series | timestamp | value | label

# Generate a Trainset compatible dataset
def TRAINSETcompatible(dataframe, variable, serieName):

    value = dataframe[variable]
    originalTimestamps = dataframe["Timestamp"]
    series = [serieName] * len(value)

    newTimestamp = list()
    t_0 = datetime.now() 
    for i in range(len(value)):
        t_i = str(t_0 + timedelta(seconds=int(originalTimestamps[i])))
        t_i = t_i.replace(" ", "T")
        t_i = t_i[:-6]
        t_i += "000Z" 
        newTimestamp.append(t_i)

    df = pd.DataFrame()
    df['series'] = series
    df["timestamp"] = newTimestamp
    df["value"] = value
    df["label"] = ""

    return df

df_x = TRAINSETcompatible(door1[0:1000000], "x", "serie_x")
df_y = TRAINSETcompatible(door1[0:1000000], "y", "serie_y")
df_z = TRAINSETcompatible(door1[0:1000000], "z", "serie_z")

In [15]:
df_x[0:250000].to_csv("to_label_datasets/x_0_250000.csv", index=False)
df_x[250000:500000].to_csv("to_label_datasets/x_250000_500000.csv", index=False)
df_x[500000:750000].to_csv("to_label_datasets/x_500000_750000.csv", index=False)
df_x[750000:1000000].to_csv("to_label_datasets/x_1000000_250000.csv", index=False)

In [42]:
df_y[0:250000].to_csv("labeled_datasets/y_0_250000.csv", index=False)
df_y[250000:500000].to_csv("labeled_datasets/y_250000_500000.csv", index=False)
df_y[500000:750000].to_csv("labeled_datasets/y_500000_750000.csv", index=False)
df_y[750000:1000000].to_csv("labeled_datasets/y_750000_1000000.csv", index=False)

In [ ]:
df_y[0:250000].to_csv("labeled_datasets/y_0_250000.csv", index=False)
df_y[250000:500000].to_csv("labeled_datasets/y_250000_500000.csv", index=False)
df_y[500000:750000].to_csv("labeled_datasets/y_500000_750000.csv", index=False)
df_y[750000:1000000].to_csv("labeled_datasets/y_1000000_250000.csv", index=False)

In [45]:
## nope
df_doble = df_x[0:250000].append(df_y[0:250000])
df_doble.to_csv("labeled_datasets/df_doble.csv", index=False)

In [40]:
df_y.head()

,series,timestamp,value,label
0,series_a,2021-12-05T10:41:51.000Z,0.517131,
1,series_a,2021-12-05T10:42:31.000Z,0.421366,
2,series_a,2021-12-05T10:43:11.000Z,0.344754,
3,series_a,2021-12-05T10:43:51.000Z,0.344754,
4,series_a,2021-12-05T10:44:30.000Z,0.325601,


### import

In [16]:
df_x_labeled = pd.DataFrame()
df_x_1 = pd.read_csv("labeled_datasets/x/x_0_250000-labeled.csv")
df_x_2 = pd.read_csv("labeled_datasets/x/x_250000_500000-labeled.csv")
df_x_3 = pd.read_csv("labeled_datasets/x/x_500000_750000-labeled.csv")
df_x_4 = pd.read_csv("labeled_datasets/x/x_750000_1000000-labeled.csv")

df_x_labeled = df_x_1.append(df_x_2).append(df_x_3).append(df_x_4)
x_label = list(df_x_labeled["label"])
x_label = [1 if x == "label_1" else 0 for x in x_label]

In [20]:
download = door1[0:1000000]
download["x_label"] = x_label

<ipython-input-20-9e4c01d1ec7a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  download["x_label"] = x_label


In [25]:
download.to_csv("labeled_door1.csv", index=False)